In [2]:
import os
import sys
import json

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
import findspark 

findspark.init()
findspark.find()
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType, StringType, StructField, IntegerType, TimestampType, DoubleType
from pyspark.sql.functions import from_json, col, udf, trim, regexp_replace, when, count,max,min,avg,sum,countDistinct, rank
from pyspark import SparkContext, SparkConf
spark = SparkSession.builder \
    .appName("join Data") \
    .getOrCreate()

In [3]:

relative = "./data/AGUAS TEST.json"
archivo_json = "./data/ABASOLO.json"

with open(archivo_json, 'r', encoding='utf-8') as archivo:
    # Carga el contenido del archivo JSON en una variable
    datos_json = json.load(archivo)

print(datos_json[0])

# for i in datos_json:
#     print(i)


{'id': '3a853c04-9812-441d-ad61-a8c65f7543d1', 'metadata': {'actualizacion': '2014-12-31T00:00:00Z', 'institucion': '', 'tipo': 'MODIFICACIÓN', 'declaracionCompleta': False, 'actualizacionConflictoInteres': False}, 'declaracion': {'situacionPatrimonial': {'datosGenerales': {'nombre': 'CESAR VICENTE', 'primerApellido': 'GARCIA', 'segundoApellido': 'GUERRERO'}, 'datosCurricularesDeclarante': {'escolaridad': [{'tipoOperacion': 'SIN_CAMBIOS', 'nivel': {'clave': 'PRI', 'valor': 'PRIMARIA'}, 'institucionEducativa': {'nombre': 'RESERVADO', 'ubicacion': 'MX'}, 'estatus': 'FINALIZADO', 'documentoObtenido': 'CERTIFICADO', 'fechaObtencion': '2017-11-29'}], 'aclaracionesObservaciones': ''}, 'datosEmpleoCargoComision': {'nivelOrdenGobierno': 'ESTATAL', 'nombreEntePublico': '', 'empleoCargoComision': 'CONTRALOR INTERNO', 'nivelEmpleoCargoComision': 'CON', 'domicilioMexico': {'municipioAlcaldia': {'clave': '001', 'valor': 'ABASOLO'}, 'entidadFederativa': {'clave': '05', 'valor': 'COAHUILA DE ZARAGOZA

In [5]:
carpeta = "./data/"

archivos_json = [os.path.join(carpeta, archivo) for archivo in os.listdir(carpeta) if archivo.endswith('.json')]
df_final = None

# Iterar sobre cada archivo JSON y unirlo al DataFrame final
datos_final = []
datos_total = 0
for e in archivos_json:
    with open(e, 'r', encoding='utf-8') as archivo:
    # Carga el contenido del archivo JSON en una variable
        datos_json = json.load(archivo)
    datos_total += len(datos_json)
    print(f"{e} : {len(datos_json)} // {datos_total}")
    for i in datos_json:
        datos_final.append(i)

with open('declaraciones_totales.json', 'w', encoding='utf-8') as file:
    json.dump(datos_final, file, ensure_ascii=False, indent=4)


./data/ABASOLO.json : 349 // 349
./data/ACUÑA.json : 3892 // 4241
./data/ADMINISTRACION FISCAL GENERAL.json : 2739 // 6980
./data/AGUAS TEST.json : 2 // 6982
./data/AGUAS Y SANEAMIENTO SABINAS.json : 18 // 7000
./data/ALLENDE.json : 874 // 7874
./data/ARCHIVO GENERAL DEL ESTADO.json : 61 // 7935
./data/ARTEAGA.json : 1356 // 9291
./data/AUDITORIA SUPERIOR DEL ESTADO DE COAHUILA.json : 1488 // 10779
./data/CANDELA.json : 139 // 10918
./data/CASTAÑOS.json : 250 // 11168
./data/CENTRO CULTURAL VITO ALESSIO ROBLES.json : 15 // 11183
./data/CENTRO DE JUSTICIA Y EMPODERAMIENTO PARA LAS MUJERES.json : 257 // 11440
./data/CERTTUC.json : 125 // 11565
./data/COLEGIO DE BACHILLERES DE COAHUILA.json : 1966 // 13531
./data/COLEGIO DE EDUCACION PROFESIONAL TECNICA DEL ESTADO DE COAHUILA.json : 553 // 14084
./data/COLEGIO DE ESTUDIOS CIENTIFICOS Y TECNOLOGICOS.json : 2363 // 16447
./data/COMISION COAHUILENSE DE CONCILIACION ARBITRAJE MEDICO.json : 14 // 16461
./data/COMISION COAHUILENSE DE CONCILIACI

In [22]:

with open("declaraciones.json", 'r', encoding='utf-8') as archivo:
    # Carga el contenido del archivo JSON en una variable
        datos_dependencias = json.load(archivo)

print(datos_dependencias)

[{'clasificador': 'Desconocido', 'dependencia': 'COORDINACION REGIONAL DE IDENTIFICACIÓN HUMANA', 'total': '10', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=204&q2=204'}, {'clasificador': 'Desconocido', 'dependencia': 'SEGURO POPULAR', 'total': '37', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=168&q2=168'}, {'clasificador': 'Desconocido', 'dependencia': 'SIN DEPENDENCIA', 'total': '4', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=0&q2=0'}, {'clasificador': 'Desconocido', 'dependencia': '', 'total': '16', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=SEDER&q2=SEDER'}, {'clasificador': 'Desconocido', 'dependencia': ' SELECCIONAR DEPENDENCIA', 'total': '34', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=16&q2=16'}, {'clasificador': 'UAdeC', 'dependencia': 'UNIVERSIDAD AUTONOMA DE COAHUILA', 'total': '9', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=200&q2=200

In [46]:
df.select("declaracion").show()

+--------------------+
|         declaracion|
+--------------------+
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
|{{{, [], true}, {...|
+--------------------+
only showing top 20 rows



df = spark.read.option("multiline", "true").format("json").load()

In [10]:
archivo = "declaraciones_totales.json"
df = spark.read.option("multiline","true").format("json").load(archivo)

df.printSchema()



root
 |-- adicional: struct (nullable = true)
 |    |-- datos_curriculares: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- aclaracionesObservaciones: string (nullable = true)
 |    |    |    |-- ambitoPublico: string (nullable = true)
 |    |    |    |-- ambitoSector: struct (nullable = true)
 |    |    |    |    |-- clave: string (nullable = true)
 |    |    |    |    |-- valor: string (nullable = true)
 |    |    |    |-- areaAdscripcion: string (nullable = true)
 |    |    |    |-- empleoCargoComision: string (nullable = true)
 |    |    |    |-- escolaridad: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- documentoObtenido: string (nullable = true)
 |    |    |    |    |    |-- estatus: string (nullable = true)
 |    |    |    |    |    |-- fechaObtencion: string (nullable = true)
 |    |    |    |    |    |-- institucionEducativa: struct (nullable = true)
 |    | 

In [23]:
df.select("declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelOrdenGobierno").show()


+------------------+
|nivelOrdenGobierno|
+------------------+
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
|           ESTATAL|
+------------------+
only showing top 20 rows



In [26]:
df.select("declaracion.situacionPatrimonial.datosEmpleoCargoComision.nombreEntePublico").show()

+-----------------+
|nombreEntePublico|
+-----------------+
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
|                 |
+-----------------+
only showing top 20 rows



In [ ]:
datos_filtro = datos_dependencias

df.filter

In [27]:

with open("declaraciones.json", 'r', encoding='utf-8') as archivo:
    # Carga el contenido del archivo JSON en una variable
        datos_dependencias = json.load(archivo)

print(datos_dependencias)


with open("declaraciones.json", 'r', encoding='utf-8') as archivo:
    # Carga el contenido del archivo JSON en una variable
        datos_transformar= json.load(archivo)

[{'clasificador': 'Desconocido', 'dependencia': 'COORDINACION REGIONAL DE IDENTIFICACIÓN HUMANA', 'total': '10', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=204&q2=204'}, {'clasificador': 'Desconocido', 'dependencia': 'SEGURO POPULAR', 'total': '37', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=168&q2=168'}, {'clasificador': 'Desconocido', 'dependencia': 'SIN DEPENDENCIA', 'total': '4', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=0&q2=0'}, {'clasificador': 'Desconocido', 'dependencia': '', 'total': '16', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=SEDER&q2=SEDER'}, {'clasificador': 'Desconocido', 'dependencia': ' SELECCIONAR DEPENDENCIA', 'total': '34', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=16&q2=16'}, {'clasificador': 'UAdeC', 'dependencia': 'UNIVERSIDAD AUTONOMA DE COAHUILA', 'total': '9', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=200&q2=200

In [31]:
municipales = []
for e in datos_dependencias:
    if e["clasificador"] == "MUNICIPIOS":
        print(e["dependencia"])
    

ABASOLO
ACUÑA
ALLENDE
ARTEAGA
CANDELA
CASTAÑOS
CUATROCIENEGAS
ESCOBEDO
FRANCISCO I. MADERO
FRONTERA
GENERAL CEPEDA
GUERRERO
HIDALGO
JIMENEZ
JUAREZ
LAMADRID
MATAMOROS
MONCLOVA
MORELOS
MUZQUIZ
NADADORES
NAVA
OCAMPO
PARRAS
PIEDRAS NEGRAS
PROGRESO
RAMOS ARIZPE
SABINAS
SACRAMENTO
SALTILLO
SAN BUENAVENTURA
SAN JUAN DE SABINAS
SAN PEDRO
SIERRA MOJADA
TORREON
VIESCA
VILLA UNION
ZARAGOZA
